In [1]:
import cPickle as pickle
import collections
import math

import plotly 
plotly.tools.set_credentials_file(username='tushar_n', api_key='5hlpvvgqia')

def flatten(l):
	return [item for sublist in l for item in sublist]


#{filename: set([obj1, obj2...])}
fl_obj_map=pickle.load(open('obj_labels_imagenet_grounded.pkl','rb'))

# remove all files that are in train
with open('train.txt', 'r') as f:
	train_files=f.read().strip().split('\n')
	for fl in train_files:
		if fl in fl_obj_map:
			fl_obj_map.pop(fl)

#{obj: {act1:[file1, file2...], act2:[], act3:[file4]}
obj_fl_map=collections.defaultdict(dict)
for fl in fl_obj_map:
	act=fl.split('_')[0]

	for obj in fl_obj_map[fl]:
		
		if act not in obj_fl_map[obj]:
			obj_fl_map[obj][act]=set()

		obj_fl_map[obj][act].add(fl)


#retrieve the set of all objects present in the files
obj_set=set.union(*[fl_obj_map[fl] for fl in fl_obj_map])

#{action: [fl1, fl2...]}
act_fl_map=collections.defaultdict(list)
for fl in fl_obj_map:
	act_fl_map[fl.split('_')[0]].append(fl)
activity_list=sorted(act_fl_map.keys())

#{action: [(obj1, count, [files])...}
obj_stats={}
for act in activity_list:
	objs=flatten([list(fl_obj_map[fl]) for fl in act_fl_map[act]])
	obj_count=collections.Counter(objs).items()
	obj_count=[(x[0], x[1], list(obj_fl_map[x[0]][act])) for x in obj_count]
	obj_count=[(x[0], x[1], ', '.join(x[2])) for x in obj_count]
	obj_count=sorted(obj_count, key=lambda x: -x[1])
	obj_stats[act]=obj_count
	
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode()

	
data=[]
for act in sorted(obj_stats.keys()):
	
	try:
		objs, counts, files=zip(*obj_stats[act])
	except:
		continue

	trace = Bar(
		x=objs,
		y=counts,
		text=files,
		marker=dict(
			color='rgb(158,202,225)',
			line=dict(
				color='rgb(8,48,107)',
				width=1.5,
			)
		),
		opacity=0.6,
		visible='legendonly',
		name=act
	)
	data.append(trace)


layout = Layout(
	title='Object Distributions',
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='Object Distributions')